In [9]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output

csvdata = pd.read_csv("SAIL2025_LVMA_data_3min_20August-25August2025_flow.csv")

data = (
    csvdata
    .assign(timestamp=lambda data: pd.to_datetime(data["timestamp"], format="%Y-%m-%d %H:%M:%S%z"))
    .sort_values(by="timestamp")
)

data = data.head(10)
print(data)

longdata = pd.melt(
    data.iloc[:,:5],
    id_vars=["timestamp"],   # Columns to keep
    var_name="Sensor",       # Name of the new 'variable' column
    value_name="Count"       # Name of the new 'value' column
).sort_values(by="timestamp")

#print(data)
print(longdata)

                  timestamp  CMSA-GAKH-01_0  CMSA-GAKH-01_180  \
0 2025-08-20 00:00:00+02:00              15                 4   
1 2025-08-20 00:03:00+02:00               1                 3   
2 2025-08-20 00:06:00+02:00               5                 4   
3 2025-08-20 00:09:00+02:00               4                 4   
4 2025-08-20 00:12:00+02:00               4                11   
5 2025-08-20 00:15:00+02:00               5                 2   
6 2025-08-20 00:18:00+02:00               3                 2   
7 2025-08-20 00:21:00+02:00              15                 6   
8 2025-08-20 00:24:00+02:00               1                 2   
9 2025-08-20 00:27:00+02:00               5                 8   

   CMSA-GAWW-11_120  CMSA-GAWW-11_300  CMSA-GAWW-12_115  CMSA-GAWW-12_295  \
0                29                33                44                28   
1                21                29                34                39   
2                35                22                

In [10]:
import plotly.express as px

fig = px.bar(
    longdata,
    x="timestamp",              # X-axis
    y="Sensor", 
    color = "Sensor", # Y-axis
    animation_frame=longdata["timestamp"].dt.strftime('%Y-%m-%d %H:%M:%S'),  # Convert datetime to string for animation
    animation_group = "Sensor",
    range_y=[0, longdata["Count"].max() + 5]  # Y-axis range
)